In [1]:
import pandas as pd
from functools import reduce
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from itertools import product

pd.options.display.max_columns = None

# labeling import
from sklearn.preprocessing import LabelEncoder

In [2]:
def preprocessing(data):
    ## 구글폼 csv 파일 불러오기
    df = pd.read_csv(data)
    
    ## mbti 20개 질문, 답변 변환
    # 라벨링 인코더 생성
    encoder = LabelEncoder()

    # 20문항 따로 데이터프레임
    quiz_df = df.iloc[:, 5:25]
    quiz_df.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

    ## labeling 
    for i in range(1, 21):
        quiz = quiz_df[i]
        encoder.fit(quiz)
        quiz2 = encoder.transform(quiz)
        quiz_df[i] = quiz2

    # +1: 1인 컬럼값들을 +1로, 0인 컬럼값들을 -1로
    for i in [1, 9, 6, 10, 18, 3, 15, 19, 8, 12]:
        quiz_df[i][quiz_df[i].values == 1] = 1
        quiz_df[i][quiz_df[i].values == 0] = -1

    # -1: 1인 컬럼값들을 -1로, 0인 컬럼값들을 +1로
    for j in [5, 13, 17, 2, 14, 7, 11, 4, 16, 20]:
        quiz_df[j][quiz_df[j].values == 1] = -1
        quiz_df[j][quiz_df[j].values == 0] = 1

    # MBTI: '모름'의 EI, SN, FT, JP 파악 (양수 or 음수)
    # EI, SN, FT, JP 컬럼 생성
    quiz_df['EI'] = 0
    quiz_df['SN'] = 0
    quiz_df['FT'] = 0
    quiz_df['JP'] = 0
    
    # 4개씩 5set로 구성된 문제들 분류
    for i in range(1, 21):
        if i % 4 == 1:
            quiz_df['EI'] += quiz_df[i]
        elif i % 4 == 2:
            quiz_df['SN'] += quiz_df[i]
        elif i % 4 == 3:
            quiz_df['FT'] += quiz_df[i]
        elif i % 4 == 0:
            quiz_df['JP'] += quiz_df[i]

    nombti = quiz_df.loc[:, 'EI':'JP']
    
    # 혹시 모를 상황 대비 copy
    nombti2 = nombti.copy()

    ## EI, SN, FT, JP 계산값들 str으로
    for i in range(len(nombti2.index)):
        for j in ['EI','SN','FT','JP']:
            if nombti2.loc[i, j] > 0: # E / S / F / J => 양수
                if j == 'EI':
                    nombti2.loc[i, j] = 'E'
                elif j == 'SN':
                    nombti2.loc[i, j] = 'S'
                elif j == 'FT':
                    nombti2.loc[i,j] = 'F'
                elif j == 'JP':
                    nombti2.loc[i, 'JP'] = 'J'
            else: # I / N / T / P => 음수
                if j == 'EI':
                    nombti2.loc[i, j] = 'I'
                elif j == 'SN':
                    nombti2.loc[i, j] = 'N'
                elif j == 'FT':
                    nombti2.loc[i,j] = 'T'
                elif j == 'JP':
                    nombti2.loc[i, 'JP'] = 'P'

    # 앞쪽 컬럼값들(str) 더해서 mbti 판단
    nombti2['mbti_guess'] = nombti2.iloc[:, 0] + nombti2.iloc[:, 1] + nombti2.iloc[:, 2] + nombti2.iloc[:, 3]
    
    # mbti_df 데이터프레임 생성
    mbti_df = df.rename(columns={'1. 당신의 MBTI는 무엇인가요?':'mbti',
                             '2. 당신의 성별은?':'sex',
                             '3. 당신의 연령대는?':'age',
                             '4. 당신의 직업은?':'job',
                             '1. 인테리어에 얼마나 관심이 있으신가요?':'like',
                             '3. 집을 볼 때 가장 중요시하는 공간은 어디인가요?':'space',
                             '4. 가장 좋아하는 색깔톤은 무엇인가요?':'color',
                             '5. 인테리어에서 가장 중요하다 생각하는 것은 무엇인가요?':'miv',
                             '모던':'modern',
                             '북유럽': 'nor_eur', 
                             '내추럴': 'natural', 
                             '빈티지': 'vintage', 
                             '프로방스 / 로맨틱': 'provence', 
                             '클래식 / 앤틱': 'classic', 
                             '한국 / 아시아': 'asia', 
                             '유니크': 'unique', 
                             'MIX': 'mix'})

    # 직업 기타 답변 처리
    # job = ['미성년자', '대학생', '직장인', '프리랜서', '사업가', '전업주부', '무직']
    mbti_df.job.replace(['ㅅㄱ', '공시생', '군인', '무직', '사회복무요원', '취준', '취준생', '퇴직자'], '무직', inplace=True)
    mbti_df.job.replace(['애견유치원선생', '유통업', '의료인', '전문직', '직장인'], '직장인', inplace=True)
    mbti_df.job.replace(['대학생', '대학원생', '대학 휴학 군 복무중'], '대학생', inplace=True)
    mbti_df.job.replace(['중고등학생'], '미성년자', inplace=True)

    # one-hot encoding
    color = pd.get_dummies(mbti_df['color']) # 선호 색깔톤
    sex = pd.get_dummies(mbti_df['sex']) # 성별
    age = pd.get_dummies(mbti_df['age']) # 나이
    job = pd.get_dummies(mbti_df['job']) # 직업

    # 필요없는, 원핫인코딩된, mbti 질문 columns 삭제
    del(mbti_df['miv'])
    del(mbti_df['color'])
    del(mbti_df['space'])
    del(mbti_df['타임스탬프'])
    del(mbti_df['2. 당신에게 집이란?'])
    del(mbti_df['sex'])
    del(mbti_df['age'])
    del(mbti_df['job'])
    mbti_df = mbti_df.drop(columns = mbti_df.loc[:, '1. 혼자 여행을 가기로 해서 숙소를 잡았다. 그 숙소는?':'20. 팀플에서 조장이 되었을 때 나는?'])
    
    # 필요한 컬럼들 병합
    mbti_df = pd.concat([mbti_df, color, sex, age, job, nombti2],axis=1)

    # mbti모르는 사람들의 mbti 추가해서 최종 mbti column 생성
    mbti_df['real_mbti'] = mbti_df['mbti']
    mbti_df['real_mbti'][mbti_df['real_mbti']=='모름'] = mbti_df['mbti_guess']

    # 응답자들의 최종 mbti 컬럼 생성
    mbti_df['real_EI'] = ''
    mbti_df['real_SN'] = ''
    mbti_df['real_FT'] = ''
    mbti_df['real_JP'] = ''

    # EI, SN, FT, JP에서 하나씩 추출
    for i in range(len(mbti_df.index)):
        mbti_df['real_EI'][i] = mbti_df['real_mbti'].values[i][0]
        mbti_df['real_SN'][i] = mbti_df['real_mbti'].values[i][1]
        mbti_df['real_FT'][i] = mbti_df['real_mbti'].values[i][2]
        mbti_df['real_JP'][i] = mbti_df['real_mbti'].values[i][3]

    # 불필요한 columns 삭제
    del(mbti_df['mbti_guess'])
    del(mbti_df['EI'])
    del(mbti_df['SN'])
    del(mbti_df['FT'])
    del(mbti_df['JP'])

    # mbti 8개 원핫인코딩
    ei = pd.get_dummies(mbti_df['real_EI'])
    sn = pd.get_dummies(mbti_df['real_SN'])
    ft = pd.get_dummies(mbti_df['real_FT'])
    jp = pd.get_dummies(mbti_df['real_JP'])

    mbti_df = pd.concat([mbti_df, ei, sn, ft, jp], axis=1)
    
    return mbti_df

In [3]:
preprocessing('MBTIxINTERIOR.csv')

,mbti,like,modern,nor_eur,natural,vintage,provence,classic,asia,unique,mix,모노톤,우드톤,웜톤,쿨톤,파스텔톤,남자,여자,10대,20대,30대,40대,50대,60대 이상,대학생,무직,미성년자,사업가,전업주부,직장인,프리랜서,real_mbti,real_EI,real_SN,real_FT,real_JP,E,I,N,S,F,T,J,P
0,INTP,1,5,3,3,3,4,3,5,2,3,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,INTP,I,N,T,P,0,1,1,0,0,1,0,1
1,INFP,4,5,4,3,1,1,2,3,1,4,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,INFP,I,N,F,P,0,1,1,0,1,0,0,1
2,ISTP,5,5,4,3,1,1,1,3,1,4,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,ISTP,I,S,T,P,0,1,0,1,0,1,0,1
3,INFJ,4,5,3,4,1,1,1,5,1,2,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,INFJ,I,N,F,J,0,1,1,0,1,0,1,0
4,ESTP,5,5,3,4,1,2,1,4,1,2,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,ESTP,E,S,T,P,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,ISTP,2,5,3,2,1,3,2,2,3,2,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,ISTP,I,S,T,P,0,1,0,1,0,1,0,1
570,ISTP,1,4,4,4,4,3,4,3,2,3,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,ISTP,I,S,T,P,0,1,0,1,0,1,0,1
571,INTP,3,4,3,3,4,4,5,5,2,2,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,INTP,I,N,T,P,0,1,1,0,0,1,0,1
572,ISTP,4,5,2,2,1,2,3,4,4,3,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,ISTP,I,S,T,P,0,1,0,1,0,1,0,1


In [ ]:
def exportcsv(df):
    df.to_csv('preprocessed.csv')